# Convolutional neural networks (CNN) for CIFAR-10/100 using PyTorch

Markus Enzweiler, markus.enzweiler@hs-esslingen.de

This is a demo used in a Computer Vision & Machine Learning lecture. Feel free to use and contribute.

We build and train a CNN for CIFAR-10 / CIFAR-100 image classification, see https://www.cs.toronto.edu/~kriz/cifar.html. We use the Python code from https://github.com/menzHSE/torch-cifar-10-cnn.git and execute it via this notebook. 

## Setup

Adapt `packagePath` to point to the directory containing this notebeook.

In [1]:
# Imports
import sys
import os
import subprocess

In [2]:
# Package Path
package_path = "./" # local
print(f"Package path: {package_path}")

Package path: ./


In [3]:
# Clone git repository

# Absolute path of the repository directory
repo_dir = os.path.join(package_path, "torch-cifar-10-cnn")
repo_url = "https://github.com/menzHSE/torch-cifar-10-cnn.git"

# Store the original working directory
original_cwd = os.getcwd()

# Check if the directory already exists using the absolute path
if os.path.exists(os.path.join(original_cwd, repo_dir)):
    print("Repository exists. Resetting to HEAD...")
    # Navigate into the repository directory
    os.chdir(repo_dir)
    # Fetch the latest changes from the remote
    subprocess.run(["git", "fetch", "origin"])
    # Reset the local branch to the latest commit from the remote
    subprocess.run(["git", "reset", "--hard", "origin/HEAD"])
    # Change back to the original working directory
    os.chdir(original_cwd)
else:
    print("Cloning repository...")
    # Clone the repository if it doesn't exist
    subprocess.run(["git", "clone", repo_url, repo_dir])


Repository exists. Resetting to HEAD...
HEAD is now at 11e6d3f Update README.md


In [4]:
# Install requirements in the current Jupyter kernel
req_file = os.path.join(repo_dir, "requirements.txt")
if os.path.exists(req_file):
    !{sys.executable} -m pip install -r {req_file}
else:
    print(f"Requirements file not found: {req_file}")

## Functions to interface with the code in the repository

In [5]:
def execute(script_name, params=None):
    script_path = os.path.join(repo_dir, script_name)
    if os.path.exists(script_path):
        print(f"Executing script: {script_path}")
        # Create the command list starting with Python and the script path
        command = ["python", script_path]
        # Add additional arguments from the params dictionary
        if params:
            for key, value in params.items():
                command.append(f"--{key}")
                command.append(str(value))
        print(command)
        subprocess.run(command)
    else:
        print(f"Script not found: {script_path}")

In [6]:
# Let's see what we can do with train.py
execute("train.py", {"help": None})

Executing script: ./torch-cifar-10-cnn/train.py
['python', './torch-cifar-10-cnn/train.py', '--help', 'None']
usage: Train a simple CNN on CIFAR-10 / CIFAR_100 with PyTorch.
       [-h] [--cpu] [--seed SEED] [--batchsize BATCHSIZE] [--epochs EPOCHS]
       [--lr LR] [--dataset {CIFAR-10,CIFAR-100}]

optional arguments:
  -h, --help            show this help message and exit
  --cpu                 Use CPU instead of GPU (cuda/mps) acceleration
  --seed SEED           Random seed
  --batchsize BATCHSIZE
                        Batch size for training
  --epochs EPOCHS       Number of training epochs
  --lr LR               Learning rate
  --dataset {CIFAR-10,CIFAR-100}
                        Select the dataset to use (CIFAR-10 or CIFAR-100)


# Train and test CNN on CIFAR-10



## Parameters

In [7]:
# parameters
batchsize = 32
seed      = 42
lr        = 3e-4
epochs    = 30
dataset   = "CIFAR-10"

## Train

In [15]:
params = {
    "dataset": dataset,           # dataset name
    "batchsize": batchsize,       # batch size
    "seed": seed,                 # random seed
    "lr": lr,                     # learning rate
    "epochs": epochs              # number of epochs     
}

# Execute 'train.py' with parameters
execute("train.py", params=params)

Executing script: ./torch-cifar-10-cnn/train.py
['python', './torch-cifar-10-cnn/train.py', '--dataset', 'CIFAR-10', '--batchsize', '32', '--seed', '42', '--lr', '0.0003', '--epochs', '30']
Using device: mps
Options: 
  Device: GPU
  Seed: 42
  Batch size: 32
  Number of epochs: 30
  Learning rate: 0.0003
  Dataset: CIFAR-10
Files already downloaded and verified
Files already downloaded and verified


/Users/menzweil/Development/miniforge3/envs/pytorch-m1-2023-10/lib/python3.9/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),


Layer (type (var_name))                  Output Shape              Param #
CNN (CNN)                                [1, 10]                   --
├─Conv2d (conv1)                         [1, 32, 32, 32]           896
├─Conv2d (conv2)                         [1, 32, 32, 32]           9,248
├─MaxPool2d (pool)                       [1, 32, 16, 16]           --
├─Dropout (drop)                         [1, 32, 16, 16]           --
├─Conv2d (conv3)                         [1, 64, 16, 16]           18,496
├─Conv2d (conv4)                         [1, 64, 16, 16]           36,928
├─MaxPool2d (pool)                       [1, 64, 8, 8]             --
├─Dropout (drop)                         [1, 64, 8, 8]             --
├─Conv2d (conv5)                         [1, 128, 8, 8]            73,856
├─Conv2d (conv6)                         [1, 128, 8, 8]            147,584
├─MaxPool2d (pool)                       [1, 128, 4, 4]            --
├─Dropout (drop)                         [1, 128, 4, 4]         

[Epoch   0] :  ......

KeyboardInterrupt: 

/Users/menzweil/Development/miniforge3/envs/pytorch-m1-2023-10/lib/python3.9/multiprocessing/resource_tracker.py:216: UserWarning: resource_tracker: There appear to be 14 leaked semaphore objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


## Test 

In [ ]:
# parameters
params = {
    "model": f"model_{dataset}_{epochs-1:03d}.pth" # model name    
}

# Execute 'train.py' with parameters
execute("test.py", params=params)

# Train and test CNN on CIFAR-100

## Parameters

In [ ]:
# parameters
batchsize = 32
seed      = 42
lr        = 3e-4
epochs    = 30
dataset   = "CIFAR-100"

## Train

In [ ]:
params = {
    "dataset": dataset,           # dataset name
    "batchsize": batchsize,       # batch size
    "seed": seed,                 # random seed
    "lr": lr,                     # learning rate
    "epochs": epochs              # number of epochs
}

# Execute 'train.py' with parameters
execute("train.py", params=params)

## Test

In [ ]:
# parameters
params = {
    "dataset": dataset,           # dataset name
    "model": f"model_{dataset}_{epochs-1:03d}.pth" # model name    
}

# Execute 'train.py' with parameters
execute("test.py", params=params)